In [1]:
import argparse
import re #regular expressions
from Bio import Entrez, SeqIO
Entrez.email = 'sethfrazer.edu'
from skbio import TabularMSA
from skbio import Protein
from skbio.alignment import global_pairwise_align_protein
import itertools

Making Function for mutagenesis block... 

In [3]:
def getAcc(accession, check):
    if accession == "manual":

        if check == 0:
            manual = input("Enter Reference Sequence: ")
        else:
            manual = input("Enter Target Sequence: ")

        return(manual)
    
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "gb")
    handle.close()
    for i,feature in enumerate(record.features):
         if feature.type=='CDS':
              aa = feature.qualifiers['translation'][0]
    return(aa)

In [13]:
def getCombos(mut_list, wt_seq, mut_acc_file):
    wt_seq = f'{wt_seq}_'
    x = 1
    mut_combos = []
    while x in range(len(mut_list)+1):
        combinations = list(itertools.combinations(mut_list, x))

        mut_accs = []
        for combo in combinations:
            acc = wt_seq

            check = 1
            for i in combo:
                if check == x:    
                    acc+=i
                else:
                    acc = f'{acc}{i},'
                check+=1
            print(acc)

            mut_combos.append(acc)
        x +=1
    print(mut_combos)
    print(len(mut_combos))

    with open(mut_acc_file, 'w') as f:
        for mut_acc in mut_combos:
            f.write(f'{mut_acc}\n')
    
    return(mut_combos)

In [5]:
def getMut(mut_combos,raccession,output_file):
    switch = 0
    bovine = Protein(getAcc(raccession, switch))
    switch+=1

    try:
        mut_combos = open(mut_combos).readlines()
    except:
        pass


    #Fetch sequences to manipulate and align
    for lines in mut_combos:     
        accession = f'{lines.split("_")[0]}_{lines.split("_")[1]}'
        mutation = lines.split('_')[2]

        wt = Protein(getAcc(accession, switch))

        substitution_matrix = substitution_matrices.load("BLOSUM45")

        ##Simple example for testing
        #wt=Protein("ABCDEF")
        #bovine=Protein("ABCDEF")

        alignment, score, start_end_positions = global_pairwise_align_protein(bovine, wt, substitution_matrix=substitution_matrix)
        dic = alignment.to_dict()
        aligned_bovine = dic[0]
        aligned_wt = dic[1]
        print(aligned_wt)

        #Check if single or multiple mutations entered
        mutsuite = mutation
        if "," in mutation :
            mutations = mutation.split(",")
        else :
            #declare empty list and append single mutation so later can be used as list if only one mutation
            mutations = []
            mutations.append(mutation)

        mutated = aligned_wt #need to copy original sequence and accumulate mutations
        for mutation in mutations:

            #check that mutation is formatted properly
            MutationRegex = re.compile(r'([A-Z])(\d+)([A-Z])')
            if re.match(MutationRegex, mutation):
                mu = MutationRegex.search(mutation)
                old = mu.group(1)
                mutsite = int(mu.group(2))
                new = mu.group(3)
            else :
                print("ERROR: Expecting mutation name in the format of XaY where X=old amino acid a=number Y=new amino acid")
                quit()

            #substract one to account for index starting at zero
            mutsite = mutsite-1

            #Count gaps to not count them when finding original site to change
            gaps = aligned_bovine[:mutsite].count('-')

            match = str(mutated[mutsite+gaps:mutsite+gaps+1])

            if match != old :
                raise Exception("ERROR: The mutation label does not match the sequence")
            else: 
                mutated = str(mutated[:mutsite+gaps]) + new + str(mutated[mutsite+gaps+1:]) 

        mutated = mutated.replace('-','')
        newacc = accession + "_" + mutsuite

        with open(output_file, 'a') as f:
            f.write(f'>{newacc}\n')
            f.write(f'{mutated}\n')
        print(newacc, end='\t\t')
        print(mutated)

    return('All Done!')

In [14]:
my_list = ['V87C', 'L165C', 'I256L', 'T62L', 'P215K', 'F283D']
wt = 'NM_001014890'
mut_acc_file = 'test_mut_acc.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'test_mut_seqs.fasta'
combos = getCombos(my_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file)

NM_001014890_V87C
NM_001014890_L165C
NM_001014890_I256L
NM_001014890_T62L
NM_001014890_P215K
NM_001014890_F283D
NM_001014890_V87C,L165C
NM_001014890_V87C,I256L
NM_001014890_V87C,T62L
NM_001014890_V87C,P215K
NM_001014890_V87C,F283D
NM_001014890_L165C,I256L
NM_001014890_L165C,T62L
NM_001014890_L165C,P215K
NM_001014890_L165C,F283D
NM_001014890_I256L,T62L
NM_001014890_I256L,P215K
NM_001014890_I256L,F283D
NM_001014890_T62L,P215K
NM_001014890_T62L,F283D
NM_001014890_P215K,F283D
NM_001014890_V87C,L165C,I256L
NM_001014890_V87C,L165C,T62L
NM_001014890_V87C,L165C,P215K
NM_001014890_V87C,L165C,F283D
NM_001014890_V87C,I256L,T62L
NM_001014890_V87C,I256L,P215K
NM_001014890_V87C,I256L,F283D
NM_001014890_V87C,T62L,P215K
NM_001014890_V87C,T62L,F283D
NM_001014890_V87C,P215K,F283D
NM_001014890_L165C,I256L,T62L
NM_001014890_L165C,I256L,P215K
NM_001014890_L165C,I256L,F283D
NM_001014890_L165C,T62L,P215K
NM_001014890_L165C,T62L,F283D
NM_001014890_L165C,P215K,F283D
NM_001014890_I256L,T62L,P215K
NM_001014890_I

c:\Users\safra\anaconda3\envs\vizphizdb_tools\Lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA
NM_001014890_V87C		MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMCFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA


c:\Users\safra\anaconda3\envs\vizphizdb_tools\Lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA
NM_001014890_L165C		MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMACACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA


...

c:\Users\safra\anaconda3\envs\vizphizdb_tools\Lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA
NM_001014890_L165C,I256L,T62L,P215K,F283D		MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVLVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMACACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIKLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVILMVIAFLICWLPYAGVAFYIFTHQGSDDGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA


c:\Users\safra\anaconda3\envs\vizphizdb_tools\Lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA
NM_001014890_V87C,L165C,I256L,T62L,P215K,F283D		MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVLVQHKKLRTPLNYILLNLAVADLFMCFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMACACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIKLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVILMVIAFLICWLPYAGVAFYIFTHQGSDDGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA


'All Done!'

In [1]:
import os
import subprocess
from deepBreaks.utils import load_obj
from deepBreaks.preprocessing import read_data
import joblib
import pandas as pd
import itertools

In [2]:
def getMutPred(report_dir, seq_add, seqFileName, metaDataFileName, mafft_exe, mt, seq_type, gap_threshold, top_per_mod, prediction_report_dir):

    load_top_mod = load_obj(top_per_mod)

    mut_ali = seqFileName[2:].split("/")[1]
    mut_ali = f'{mut_ali.split(".")[0]}_mutant_aligned.fasta'
    print(mut_ali)
    cmd = [mafft_exe, '--add', seq_add, '--keeplength', seqFileName , '>', f'{report_dir}/{mut_ali}' ]
    aligner = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out_put = aligner.communicate()[0].decode('utf8')

    #print(out_put)
    mut_test = read_data( f'./{report_dir}{mut_ali}', seq_type = seq_type, is_main=True, gap_threshold=gap_threshold)
    ref_copy = read_data( seqFileName, seq_type = seq_type, is_main=True, gap_threshold=gap_threshold)
    last_seq = ref_copy.shape[0]
    print(ref_copy.shape)
    #print(last_seq)
    mut_test = mut_test.iloc[last_seq:].copy()
    print(mut_test)

    predictions = load_top_mod.predict(mut_test)
    print(predictions)
    mut_fin = pd.DataFrame(index=mut_test.index)
    mut_fin['Prediction'] = predictions
    mut_fin.to_csv(path_or_buf= prediction_report_dir,mode="w")

    return('Model Predictions Complete!')

In [3]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
seq_add = 'mutant_seqs_rods.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './rod_aligned_db_fmt_Lambda_Max_2023-08-10_12-50-35/'
seqFileName = f'{report_dir}rod_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}rod_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.6
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'BayesianRidge.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}/rods_mutant_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

rod_aligned_db_fmt_mutant_aligned.fasta
(391, 354)
                                               p1 p2 p3 p4 p5 p6 p7 p8 p9 p10  \
NM_001014890_V87C                               M  N  G  T  E  G  P  N  F   Y   
NM_001014890_L165C                              M  N  G  T  E  G  P  N  F   Y   
NM_001014890_I256L                              M  N  G  T  E  G  P  N  F   Y   
NM_001014890_T62L                               M  N  G  T  E  G  P  N  F   Y   
NM_001014890_P215K                              M  N  G  T  E  G  P  N  F   Y   
...                                            .. .. .. .. .. .. .. .. ..  ..   
NM_001014890_V87C,L165C,I256L,P215K,F283D       M  N  G  T  E  G  P  N  F   Y   
NM_001014890_V87C,L165C,T62L,P215K,F283D        M  N  G  T  E  G  P  N  F   Y   
NM_001014890_V87C,I256L,T62L,P215K,F283D        M  N  G  T  E  G  P  N  F   Y   
NM_001014890_L165C,I256L,T62L,P215K,F283D       M  N  G  T  E  G  P  N  F   Y   
NM_001014890_V87C,L165C,I256L,T62L,P215K,F283D  M  N  G  T

'Model Predictions Complete!'